In [2]:
import cv2
import numpy as np

# Start webcam
cap = cv2.VideoCapture(0)

# For reducing flicker
detected_frames = 0
MIN_STABLE_FRAMES = 2

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Resize for consistency
    frame = cv2.resize(frame, (640, 480))

    # Convert to HSV color space
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # Define range for fluorescent yellow/green
    lower_yellow = np.array([25, 100, 100])
    upper_yellow = np.array([45, 255, 255])

    # Mask for yellow/green region
    mask = cv2.inRange(hsv, lower_yellow, upper_yellow)

    # Morphological filtering (reduce noise)
    kernel = np.ones((5, 5), np.uint8)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel, iterations=4)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel, iterations=7)

    # Find contours from mask
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Track if any valid detection happened this frame
    shuttle_detected = False

    for cnt in contours:
        area = cv2.contourArea(cnt)
        if area > 200:
            x, y, w, h = cv2.boundingRect(cnt)
            aspect_ratio = w / float(h)

            if 0.5 < aspect_ratio < 1.5:
                shuttle_detected = True
                stable_x, stable_y, stable_w, stable_h = x, y, w, h  # save for drawing

    # Flicker filtering logic
    if shuttle_detected:
        detected_frames += 1
    else:
        detected_frames = 0

    if detected_frames >= MIN_STABLE_FRAMES:
        cv2.rectangle(frame, (stable_x, stable_y), (stable_x + stable_w, stable_y + stable_h), (0, 255, 0), 2)
        cv2.putText(frame, "Shuttle", (stable_x, stable_y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

    # Show result
    cv2.imshow("Shuttlecock Detection", frame)

    # Exit on ESC
    if cv2.waitKey(1) == 27:
        break

cap.release()
cv2.destroyAllWindows()
